##Part 1

In [ ]:
from tensorflow.keras.datasets import imdb

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen = 400)
X_test =  pad_sequences(X_test, maxlen = 400)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((25000, 400), (25000, 400), (25000,), (25000,))

In [ ]:
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [ ]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [ ]:
index_dict = imdb.get_word_index()

In [ ]:
for enc in X_train[0]:
  for k, v in index_dict.items():
    if enc == v:
      print(k,end=' ')

In [ ]:
res = 'positive' if y_train[0]==1 else 'negative'
print(f'Sentiment of above sentence is {res}')

Sentiment of above sentence is positive


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM,TimeDistributed, Flatten, Dense

model = Sequential()
model.add(Embedding(input_dim = 10000, output_dim = 100, input_length = 400))
model.add(LSTM(32, activation = 'relu', return_sequences=True))

model.add(TimeDistributed(Dense(100)))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 400, 100)          1000000   
                                                                 
 lstm_3 (LSTM)               (None, 400, 32)           17024     
                                                                 
 time_distributed_3 (TimeDis  (None, 400, 100)         3300      
 tributed)                                                       
                                                                 
 flatten_3 (Flatten)         (None, 40000)             0         
                                                                 
 dense_14 (Dense)            (None, 512)               20480512  
                                                                 
 dense_15 (Dense)            (None, 256)               131328    
                                                      

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)

Epoch 1/5
391/391 [==============================] - 213s 545ms/step - loss: 0.2171 - accuracy: 0.9164 - val_loss: 0.2898 - val_accuracy: 0.8815
Epoch 2/5
391/391 [==============================] - 212s 543ms/step - loss: 0.1194 - accuracy: 0.9562 - val_loss: 0.3540 - val_accuracy: 0.8690
Epoch 3/5
391/391 [==============================] - 213s 545ms/step - loss: 0.0485 - accuracy: 0.9835 - val_loss: 0.5675 - val_accuracy: 0.8615
Epoch 4/5
391/391 [==============================] - 215s 551ms/step - loss: 0.0237 - accuracy: 0.9918 - val_loss: 0.7885 - val_accuracy: 0.8602
Epoch 5/5
391/391 [==============================] - 215s 550ms/step - loss: 0.0230 - accuracy: 0.9918 - val_loss: 0.6906 - val_accuracy: 0.8603


In [ ]:
score = model.evaluate(X_test, y_test)
print(f"Loss: {score[0]}, Accuracy:{score[1]}")
#sample:
model.predict(X_test[0].reshape((1, 400)))

782/782 [==============================] - 55s 70ms/step - loss: 0.6906 - accuracy: 0.8603
Loss: 0.6906431913375854, Accuracy:0.860319972038269


array([[0.00543827]], dtype=float32)

##Part 2

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
import pandas as pd
df = pd.read_json('https://raw.githubusercontent.com/rishabhmisra/News-Headlines-Dataset-For-Sarcasm-Detection/master/Sarcasm_Headlines_Dataset.json',lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [7]:
df.drop(['article_link'], axis=1,inplace=True)

In [9]:
df.len = df.headline.apply(lambda x: len(x.split(" ")))
df.head()

,is_sarcastic,headline,len
0,1,thirtysomething scientists unveil doomsday clo...,8
1,0,dem rep. totally nails why congress is falling...,13
2,0,eat your veggies: 9 deliciously different recipes,7
3,1,inclement weather prevents liar from getting t...,8
4,1,mother comes pretty close to using word 'strea...,9


In [12]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

token = Tokenizer(num_words=10000)
token.fit_on_texts(list(df.headline))
X = token.texts_to_sequences(df.headline)
print("No. of samples:", len(X))

X = pad_sequences(X, maxlen = 25)
y = np.asarray(df.is_sarcastic)
print("No. of labels: ", len(y))
print(y[0])

Number of Samples: 28619
Number of Labels:  28619
1


In [13]:
print(X)

[[   0    0    0 ...    2  660 1118]
 [   0    0    0 ... 1527 2153 1844]
 [   0    0    0 ...  261  646 1483]
 ...
 [   0    0    0 ...    6  842 1923]
 [   0    0    0 ... 2359  837 6295]
 [   0    0    0 ...    6  259  177]]


In [14]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!rm glove.6B.zip glove.6B.50d.txt glove.6B.100d.txt  glove.6B.200d.txt

--2022-04-25 18:32:56--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-04-25 18:32:56--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-04-25 18:32:56--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [18]:
embeddings = {}
for i in open('glove.6B.300d.txt'):
    w = i.split(" ")[0]
    emb = i.split(" ")[1:]
    emb = np.asarray(emb, dtype='float32')
    embeddings[w] = emb

emb_mat = np.zeros((len(token.word_index) + 1, 300))

for w, i in token.word_index.items():
	emb_vec = embeddings.get(w)
	if emb_vec is not None:
		emb_mat[i] = emb_vec

In [20]:
len(embeddings.values())

400000

In [27]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Bidirectional, Dense, Dropout, Embedding, LSTM
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model = Sequential()
model.add(Embedding(len(token.word_index) + 1, 300, weights = [emb_mat]))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss= SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

model_seq = model.fit(X, y, batch_size=100, epochs=5, validation_split=0.2)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


InvalidArgumentError: ignored